In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import altair as alt

In [2]:
df = pd.read_csv('linkedin_data_sample.csv')

In [3]:
def generate_meta(df):
    """ generates meta data for a df """

    df_meta = pd.DataFrame()
    df_meta['datatype'] = df.dtypes
    df_meta['total_values'] = [df[col].size - df[col].isna().sum() for col in df.columns]
    df_meta['null_values'] = [df[col].isna().sum() for col in df.columns]
    df_meta['distinct_values'] = [len(set(df[col])) for col in df.columns]

    return df_meta

# keep only usd
df = df[df.currency=='USD']

# explore data
df_meta = generate_meta(df)

# drop columns with all null, single value, or untelligible values
drop_cols = list(df_meta[(df_meta.distinct_values==1) | (df_meta.null_values == len(df))].index)
drop_cols.extend(['expiry', 'listed_time'])
drop_cols.extend(['expiry', 
            'listed_time', 
            'application_type',
            'application_url',
            'company_id',
            'posting_domain',
            'skills_desc'])

# drop rows without a fips (location)
df.dropna(subset=['fips'], inplace=True)
df['fips_state'] = [int(str(string)[:-5]) for string in df.fips]
            
print('dropping these columns:', drop_cols)
df.drop(drop_cols, axis=1, inplace=True)
print(df.shape)

generate_meta(df)

dropping these columns: ['pay_period', 'med_salary', 'sponsored', 'work_type', 'currency', 'compensation_type', 'expiry', 'listed_time', 'expiry', 'listed_time', 'application_type', 'application_url', 'company_id', 'posting_domain', 'skills_desc']
(9750, 17)


,datatype,total_values,null_values,distinct_values
Unnamed: 0,int64,9750,0,9750
job_id,int64,9750,0,9750
company_name,object,9750,0,3565
title,object,9750,0,7324
description,object,9750,0,9259
max_salary,float64,9750,0,2110
location,object,9750,0,1761
views,float64,9750,0,209
min_salary,float64,9750,0,1921
applies,float64,2508,7242,7334


In [4]:
job_industries = pd.read_csv('job_industries.csv')
industries = pd.read_csv('industries.csv')
df_jobs = pd.merge(job_industries, industries, on='industry_id', how='inner')
df_jobs = df_jobs.groupby('job_id')['industry_name'].agg(lambda x: list(x.unique())).reset_index()
df_jobs['first_industry'] = [ind_list[0] for ind_list in df_jobs.industry_name]

job_skills = pd.read_csv('job_skills.csv')
skills = pd.read_csv('skills.csv')
job_skills = pd.merge(job_skills, skills, on='skill_abr', how='inner')
job_skills = job_skills.groupby('job_id')['skill_name'].agg(lambda x: list(x.unique())).reset_index()

df_jobs = pd.merge(df_jobs, job_skills, on='job_id', how='inner')
df = pd.merge(df_jobs, df, on='job_id')

import geopandas 
state = geopandas.read_file('ne_110m_admin_1_states_provinces')
state['fips_state'] = [int(string[2:]) for string in state.fips]
region_state = state[['fips_state','region']]
df = pd.merge(df, region_state, on='fips_state', how='left')

industry_levels = pd.read_csv('industry_level_mapping.csv')
df = pd.merge(df, industry_levels, left_on='first_industry', right_on='Industry')

In [5]:
column_order = ['job_id', 
'company_name', 
'title', 
'location',
'region',
'first_industry', 
'industry_name', 
'skill_name',
'formatted_experience_level', 
'description',
'normalized_salary', 
'max_salary', 
'min_salary',
'views', 
'applies', 
'remote_allowed', 
'zip_code',
'fips',
'fips_state',
'job_posting_url']

df = df[column_order]
print(df.shape)
df.head()

(9663, 20)


,job_id,company_name,title,location,region,first_industry,industry_name,skill_name,formatted_experience_level,description,normalized_salary,max_salary,min_salary,views,applies,remote_allowed,zip_code,fips,fips_state,job_posting_url
0,2989631782,ActOne Group,Administrative Assistant - CONCUR,"New York, NY",Northeast,Financial Services,[Financial Services],[Administrative],Associate,Global Financial Services firm is seeking an e...,82500.0,90000.0,75000.0,1.0,NaN,NaN,10001.0,36061.0,36,https://www.linkedin.com/jobs/view/2989631782/...
1,3177010992,ABC Farigua Division,Customer Service Representative,"Greater Orlando, FL",South,Insurance Agencies and Brokerages,"[Insurance Agencies and Brokerages, Food and B...","[Customer Service, Sales]",Entry level,We are seeking future agents to join our team!...,90000.0,105000.0,75000.0,6.0,NaN,NaN,32801.0,12095.0,12,https://www.linkedin.com/jobs/view/3177010992/...
2,3619548798,"Dexterity, Inc.",Senior Mechanical Engineer,"Redwood City, CA",West,Computer Hardware Manufacturing,"[Computer Hardware Manufacturing, Software Dev...",[Engineering],Mid-Senior level,Senior Mechanical Engineer\nLocation: Redwood ...,170000.0,190000.0,150000.0,3.0,NaN,NaN,94061.0,6081.0,6,https://www.linkedin.com/jobs/view/3619548798/...
3,3681437990,Ascendion,Quality Assurance Specialist,"Seattle, WA",West,Retail,"[Retail, Food and Beverage Services, Food and ...","[Administrative, Quality Assurance]",Associate,About Ascendion Ascendion is a full-service di...,66000.0,72000.0,60000.0,8.0,NaN,NaN,98101.0,53033.0,53,https://www.linkedin.com/jobs/view/3681437990/...
4,3803052628,Insight Global,Flight Software Engineer,"Webster, TX",South,Defense and Space Manufacturing,[Defense and Space Manufacturing],[Engineering],Mid-Senior level,Must Haves:10-15 years of experience with C++ ...,155000.0,170000.0,140000.0,36.0,7.0,NaN,77598.0,48201.0,48,https://www.linkedin.com/jobs/view/3803052628/...


df.to_csv('linkedin_df.csv')